How to run a stellar evolution code
====================

Here we will generate initial conditions for a stellar evolution code. We will adopt a parametrized stellar evolution code.

For reference you can read chapter 3 of Portegies Zwart & McMillan 2018 (2018araa.book.....P). 

With this tutorial you will learn
- how to generate inital conditions (mass-function)
- how to run a parametrized stellar evolution code
- channels and data transfer
- plotting results

In [ ]:
%matplotlib inline
import numpy
from matplotlib import pyplot
from amuse.units import units

We start by setting up a simulation by specifying the initial mass function. What would you say is the biggest difference between these IMFs?

In [ ]:
from amuse.datamodel import Particles
n_stars = 1024
mmin = 0.1 | units.MSun
mmax = 100 | units.MSun

from amuse.lab import new_kroupa_mass_distribution
mkroupa = new_kroupa_mass_distribution(n_stars,
                                           mass_min=mmin, 
                                           mass_max=mmax)
k_stars = Particles(mass=mkroupa)
from amuse.lab import new_salpeter_mass_distribution
msalpeter = new_salpeter_mass_distribution(n_stars, 
                                           mass_min=mmin, 
                                           mass_max=mmax)
s_stars = Particles(mass=msalpeter)
print("mean-mass for Kroupa=", numpy.mean(k_stars.mass))
print("mean-mass for Salpeter=", numpy.mean(s_stars.mass))

pyplot.hist(numpy.log10(k_stars.mass.value_in(units.MSun)))
pyplot.title('Kroupa IMF')
pyplot.yscale('log')
pyplot.xlim(-1, 2)
pyplot.figure()
pyplot.hist(numpy.log10(s_stars.mass.value_in(units.MSun)))
pyplot.title('Salpeter IMF')
pyplot.yscale('log')
pyplot.xlim(-1, 2)
pyplot.show()

Now that the mass functions are determined, we can evolve the stars in time. For this we start two stellar evolution codes, both parametrized, and evolve the stars in time.

In [ ]:
from amuse.community.seba.interface import SeBa
def start_stellar_code(stars):
    stellar = SeBa()
    stellar.particles.add_particles(stars)
    channels = {"to_stars": stellar.particles.new_channel_to(stars), 
                "to_stellar": stars.new_channel_to(stellar.particles)}
    return stellar, channels
kstellar, kchannels = start_stellar_code(k_stars)
sstellar, schannels = start_stellar_code(s_stars)

Now the stellar evolution code are declared. Now evolve in time and calculate the mean mass as a function of time.

In [ ]:
times = 10**numpy.arange(0.0, 4.0, 0.1) | units.Myr
mmean = []
for time in times:
    kstellar.evolve_model(time)
    kchannels["to_stars"].copy()
    sstellar.evolve_model(time)
    schannels["to_stars"].copy()
    mmean.append(numpy.mean(k_stars.mass)/numpy.mean(s_stars.mass))
kstellar.stop()
sstellar.stop()
print("mean-mass for Kroupa=", numpy.mean(k_stars.mass))
print("mean-mass for Salpeter=", numpy.mean(s_stars.mass))

In [ ]:
from amuse.plot import plot
plot(times, mmean)
pyplot.ylabel("relative mean mass")
pyplot.semilogx()
pyplot.show()

In [ ]:
from amuse.plot import scatter
scatter(s_stars.temperature, s_stars.luminosity, c='r')
scatter(k_stars.temperature, k_stars.luminosity, c='b', s=3)
pyplot.xlim(2.e+4, 2000)
pyplot.ylim(1.e-5, 1000)
pyplot.loglog()
pyplot.show()

You have calculated the evolution of a stellar population with solar metalicity.

Assignments and questions:
---------------

### Assignment 1:
Run the notebook and describe the evolution of the relative mean stellar mass. Why does it drop with time?

### Assignment 2:
Compare the mean mass of two stellar populations with different metalicity, 0.02 (solar) and 0.002 (one tenth times solar) but the same stellar masses.

### Question 1:
Is there a difference in the number of compact objects (white dwarfs, neutron stars, black holes; these types are defined [here](https://github.com/amusecode/amuse/blob/main/src/amuse/units/units.py), and e.g. 1.|stellar_type corresponds to a main sequene star) in the populations with different initial mass functions, and with different metallicities, and is this difference statistically significant?

Is a similar difference also noticeable in the number of main sequence stars?

### Assignment 3:
Calculate, instead of the mean mass, the total luminosity and luminosity-weighted temperature ($T_{LW}=(\Sigma_iL_iT_i)/(\Sigma_iL_i)$, the effective color of the integrated stellar population) of the entire population of stars as a function of time.

Make a plot of these parameters for both mass functions, and both metallicities. 

### Question 4:
Can one, based on observational differences in luminosity and integrated temperature, make a distinction between the two mass functions?